In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# データの読み込み

folder_path = '../input/ieee-fraud-detection/'
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
sub = pd.read_csv(f'{folder_path}sample_submission.csv')

In [ ]:
train_transaction.head()

In [ ]:
train_identity.shape, train_transaction.shape, test_identity.shape, test_transaction.shape

In [ ]:
# test_identity = test_identity.rename(columns={
#     'id-01':'id_01','id-02':'id_02','id-03':'id_03','id-04':'id_04','id-05':'id_05','id-06':'id_06','id-07':'id_07','id-08':'id_08',
#     'id-09':'id_09','id-10':'id_10','id-11':'id_11','id-12':'id_12','id-13':'id_13','id-14':'id_14','id-15':'id_15','id-16':'id_16',
#     'id-17':'id_17','id-18':'id_18','id-19':'id_19','id-20':'id_20','id-21':'id_21','id-22':'id_22','id-23':'id_23','id-24':'id_24',
#     'id-25':'id_25','id-26':'id_26','id-27':'id_27','id-28':'id_28','id-29':'id_29','id-30':'id_30','id-31':'id_31','id-32':'id_32',
#     'id-33':'id_33','id-34':'id_34','id-35':'id_35','id-36':'id_36','id-37':'id_37','id-38':'id_38',

# })

# train_transaction = pd.merge(train_transaction,train_identity,how='left')
# test_transaction = pd.merge(test_transaction,test_identity,how='left')

In [ ]:
# ワンホットエンコーディング
onehot_cols = [
    'ProductCD',
    'card4',
    'card6',
              ]
train_transaction = pd.get_dummies(train_transaction, columns=onehot_cols, sparse=False)
test_transaction = pd.get_dummies(test_transaction, columns=onehot_cols, sparse=False)

In [ ]:
#ラベルエンコーディング

from sklearn import preprocessing
transaction = pd.concat([train_transaction.drop("isFraud", axis=1), test_transaction])[["P_emaildomain", "R_emaildomain", 
        "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9",
#          "id_12", "id_15", "id_16", "id_23", "id_27", "id_28", "id_29", "id_30", "id_31", "id_33", "id_34",
#           "id_35", "id_36", "id_37", "id_38", "DeviceType", "DeviceInfo"
        ]]
le_list= ["P_emaildomain", "R_emaildomain", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9",
#          "id_12", "id_15", "id_16", "id_23", "id_27", "id_28", "id_29", "id_30", "id_31", "id_33", "id_34",
#           "id_35", "id_36", "id_37", "id_38", "DeviceType", "DeviceInfo"
         ]

for col in le_list:
    transaction[col] = transaction[col].astype("str")
    train_transaction[col] = train_transaction[col].astype("str")
    test_transaction[col] = test_transaction[col].astype("str")
    print(col)
    le = preprocessing.LabelEncoder()
    le.fit(transaction[col])
    
    train_transaction[col] = le.transform(train_transaction[col])
    test_transaction[col] = le.transform(test_transaction[col])

In [ ]:
# drop_list = ['ProductCD','card4','card6',"P_emaildomain", "R_emaildomain", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9"]
# train_transaction = train_transaction.drop(drop_list, axis=1)
# test_transaction = test_transaction.drop(drop_list, axis=1)

In [ ]:
#不正と真正にデータをわける

y_train = train_transaction["isFraud"]
X_train = train_transaction.drop(["isFraud"
                                  , "card6_debit"
                                 ], axis=1)

X_test = test_transaction

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import mean_squared_error
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=False)

    
#lightGBMで学習


lgbm_params = {  'objective': 'binary', 
                 'num_boost_round':10000, 
                 'early_stopping_rounds':10, 
                 'learning_rate': 0.08332719243688654,
                 'scale_pos_weight': 0.10005189941581771,
                 'num_leaves': 65,
                 'max_depth': 18,
                 "metric": 'auc',
                 "verbosity": 100
              }


lgb_train = lgb.Dataset(X_train, y_train, weight = compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'))
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

model = lgb.train(lgbm_params, 
                  lgb_train, 
                  valid_sets=lgb_val)

predict_proba = model.predict(X_test, num_iteration=model.best_iteration)
pred = [0 if i < 0.5 else 1 for i in predict_proba]


In [ ]:
submission = pd.DataFrame(test_transaction["TransactionID"])
submission["isFraud"] = predict_proba

submission.to_csv('submission.csv', index=False)

In [ ]:

# import lightgbm as lgb
# from sklearn.model_selection import train_test_split
# from sklearn.utils.class_weight import compute_sample_weight
# from sklearn.metrics import mean_squared_error
# import numpy as np

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=False)

# def objective(trial):
    
# #lightGBMで学習


    

#     lgbm_params = {  'objective': 'binary', 
#                      'num_boost_round':10000, 
#                      'early_stopping_rounds':10, 
#                      'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
#                      'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 0.1, 1),
#                      'num_leaves': trial.suggest_int('num_leaves', 20, 200),
#                      'max_depth': trial.suggest_int('max_depth', 5, 20),
#                      "metric": 'auc',
#                      "verbosity": 100
#                   }


#     lgb_train = lgb.Dataset(X_train, y_train, weight = compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'))
#     lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

#     model = lgb.train(lgbm_params, 
#                       lgb_train, 
#                       valid_sets=lgb_val)

# #     predict_proba = model.predict(X_test, num_iteration=model.best_iteration)
# #     pred = [0 if i < 0.5 else 1 for i in predict_proba]
#     predict_proba = model.predict(X_val, num_iteration=model.best_iteration)
#     pred = [0 if i < 0.5 else 1 for i in predict_proba]
    
#     return np.sqrt(mean_squared_error(y_val, pred))

In [ ]:
import optuna
# if __name__ == '__main__':
#     study = optuna.create_study()
#     study.optimize(objective, n_trials=50)
#     print(study.best_trial)

In [ ]:
# submission = pd.DataFrame(test_transaction["TransactionID"])
# submission["isFraud"] = predict_proba

# submission.to_csv('submission.csv', index=False)